In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from datetime import timedelta
from dateutil import parser
import re
import matplotlib.pyplot as plt
from subprocess import check_output
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import emoji
import string
import preprocessor as p

In [3]:
tweets = pd.read_csv('../data/raw_trump_tweets.csv')

In [15]:
# Filter tweets between 20th jan 2017 and 31st Dec 2018 (as we have stock data between these two dates)
tweets['date']= pd.to_datetime(tweets['date'])
tweets = tweets[tweets['date'] >= np.datetime64('2017-01-20')]
tweets = tweets[tweets['date'] <= np.datetime64('2018-12-31')]

In [16]:
def process_and_remove_duplicates(tweets, date):
    duplicated_rows = tweets[tweets['date'] == date]
    id = max(duplicated_rows['id'])
    max_favorite = max(duplicated_rows['favorites'])
    max_retweet = max(duplicated_rows['retweets'])
    combined_tweet = ""
    device = duplicated_rows['device'].iloc[0]
    for index, row in duplicated_rows.iterrows():
        combined_tweet = row['text'] + ' ' + combined_tweet.strip('.')
    tweets = tweets[tweets.date != date]
    row = {'id': id, 'text': combined_tweet, 'isRetweet': 'f', 'isDeleted': 'f', 'device': device, 'favorites': max_favorite, 'retweets': max_retweet, 'date': date}
    tweets = tweets.append(row, ignore_index = True)   
    print(f"Combined {len(duplicated_rows)} tweets on {date}")
    return tweets

In [17]:
'''
As twitter has a 260 character limit, some tweets are split up and posted at the same time. 
We will combine such tweets.
'''
dates = tweets["date"]
duplicated = tweets[dates.isin(dates[dates.duplicated()])]
duplicated_dates = duplicated['date'].unique()

for date in duplicated_dates:
    tweets = process_and_remove_duplicates(tweets, date)

Combined 2 tweets on 2018-11-21T22:18:00.000000000
Combined 2 tweets on 2018-09-10T21:59:00.000000000
Combined 2 tweets on 2018-08-14T01:37:00.000000000
Combined 2 tweets on 2018-05-31T10:56:00.000000000
Combined 2 tweets on 2018-02-01T22:37:00.000000000
Combined 2 tweets on 2018-03-09T17:50:00.000000000
Combined 2 tweets on 2017-12-29T12:50:00.000000000
Combined 2 tweets on 2017-12-29T12:48:00.000000000
Combined 3 tweets on 2017-12-28T22:17:00.000000000
Combined 2 tweets on 2017-12-22T20:47:00.000000000
Combined 2 tweets on 2017-12-16T03:09:00.000000000
Combined 2 tweets on 2017-11-18T13:49:00.000000000
Combined 2 tweets on 2017-11-15T10:22:00.000000000
Combined 2 tweets on 2017-07-11T10:53:00.000000000
Combined 2 tweets on 2017-07-11T10:37:00.000000000
Combined 2 tweets on 2017-10-28T21:09:00.000000000
Combined 2 tweets on 2017-10-26T01:47:00.000000000
Combined 2 tweets on 2017-10-14T11:08:00.000000000
Combined 2 tweets on 2017-05-10T10:31:00.000000000
Combined 2 tweets on 2017-09-30

Combined 4 tweets on 2018-06-19T13:52:00.000000000
Combined 2 tweets on 2018-06-18T13:50:00.000000000
Combined 3 tweets on 2018-06-17T14:36:00.000000000
Combined 2 tweets on 2018-06-15T11:56:00.000000000
Combined 4 tweets on 2018-06-15T11:55:00.000000000
Combined 2 tweets on 2018-12-06T08:53:00.000000000
Combined 2 tweets on 2018-06-14T15:09:00.000000000
Combined 2 tweets on 2018-06-14T15:08:00.000000000
Combined 2 tweets on 2018-06-13T09:40:00.000000000
Combined 2 tweets on 2018-12-06T20:40:00.000000000
Combined 2 tweets on 2018-09-06T20:58:00.000000000
Combined 2 tweets on 2018-09-06T20:56:00.000000000
Combined 2 tweets on 2018-06-06T00:40:00.000000000
Combined 2 tweets on 2018-05-30T12:47:00.000000000
Combined 4 tweets on 2018-05-25T12:04:00.000000000
Combined 2 tweets on 2018-05-23T01:13:00.000000000
Combined 2 tweets on 2018-05-16T18:40:00.000000000
Combined 3 tweets on 2018-05-16T13:09:00.000000000
Combined 2 tweets on 2018-08-05T20:33:00.000000000
Combined 3 tweets on 2018-04-21

In [18]:
# Remove Retweets
tweets1 = tweets[tweets['isRetweet'] == 'f'].reset_index(drop=True)
# Remove retweets which are not labelled as retweets 
tweets1 = tweets1[tweets1['text'].str.contains('RT @') == False]

# Feauture extraction
tweets2 = tweets1.loc[:, ['id', 'text', 'favorites', 'retweets', 'date']]
tweets2['date_new'] = [i + datetime.timedelta(hours = 8) for i in tweets2['date']]
tweets2['date_part'] = [i.date() for i in tweets2['date_new']]
tweets2['time_part'] = [i.time() for i in tweets2['date_new']]
tweets2['hour'] = [int(str(i).split(":")[0]) for i in tweets2['time_part']]
tweets2['year'] = [int(str(i).split("-")[0]) for i in tweets2['date_part']]
tweets2['month'] = [int(str(i).split("-")[1]) for i in tweets2['date_part']]
tweets2 = tweets2.reset_index(drop=True)

In [25]:
# Helper functions to clean tweets according to the requirements of different models

def tweet_clean(text):
    '''
    Calls tweet preprocessing library to clean tweets 
    '''
    text1 = p.clean(text)
    return text1

def tweet_clean_sentiment_analysis(text):
    '''
    Similar to previous function, but does not clean everything. 
    '''
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.RESERVED, p.OPT.NUMBER)
    text1 = p.clean(text)
    return text1

def preprocess_data(data):
 #Removes Numbers
    data = data.astype(str).str.replace(r'\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
                
    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]

    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)



In [26]:
# Tweets cleaned for topic modelling
tweets2['text_topic_modelling'] = tweets2['text'].apply(tweet_clean)
tweets2['text_topic_modelling'] = preprocess_data(tweets2['text_topic_modelling'])
stop_words = stopwords.words('english')
tweets2['text_topic_modelling'] = tweets2['text_topic_modelling'].apply(lambda x: [item for item in x if item not in stop_words])
tweets2['text_topic_modelling'] = tweets2['text_topic_modelling'].apply(lambda x: " ".join(x))

# Tweets cleaned for EDA 
tweets2['text_EDA'] = tweets2['text']

# Tweets cleaned for sentiment analysis
tweets2['text_sentiment_analysis'] = tweets2['text'].apply(tweet_clean_sentiment_analysis)

# Tweets cleaned for LSTM models
tweets2['text_LSTM'] = tweets2['text'].apply(tweet_clean)
tweets2['text_LSTM'] = preprocess_data(tweets2['text_LSTM'])
tweets2['text_LSTM'] = tweets2['text_LSTM'].apply(lambda x: " ".join(x))


In [27]:
# Remove tweets outside trading hours 
trump_tweets = tweets2.copy()
trump_tweets['date_new'] = pd.to_datetime(trump_tweets['date_new'])
trump_tweets.rename(columns={"date_new": "tweet_datetime"}, inplace=True)
trump_tweets.set_index('tweet_datetime', inplace=True, drop=False)
trump_tweets.index.name = None
trump_tweets = trump_tweets.between_time(datetime.time(4), datetime.time(20), include_start=True, include_end=True)

### Pre Processing of Stock Data

In [29]:
# Datasets are combined after pre processing! (else kernel crashes)
stock_price_2017 = pd.read_csv('../data/yr2017.csv')
stock_price_2018 = pd.read_csv('../data/yr2018.csv')

In [ ]:
# Drop unwanted columns 
stock_price_2017.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)
stock_price_2018.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)

In [ ]:
# Change to datetime objects
stock_price_2017['TIME_M'] = pd.to_datetime(stock_price_2017['TIME_M'], format='%H:%M:%S.%f')
stock_price_2018['TIME_M'] = pd.to_datetime(stock_price_2018['TIME_M'], format='%H:%M:%S.%f')

In [ ]:
# Remove seconds, milliseconds from data
stock_price_2017['TIME_M'] = stock_price_2017['TIME_M'].dt.floor('Min').dt.time
stock_price_2018['TIME_M'] = stock_price_2018['TIME_M'].dt.floor('Min').dt.time

In [ ]:
# Get minute to minute data (one entry for each minute)
stock_price_2017.drop_duplicates(subset=['TIME_M','DATE'],inplace=True)
stock_price_2018.drop_duplicates(subset=['TIME_M', 'DATE'],inplace=True)

In [122]:
# Export to csv, will be used for preprocessing later!
stock_price = pd.concat([stock_price_2017,stock_price_2018], ignore_index=True, sort=False)
stock_price.to_csv("../data/stock_price.csv", index=False, header=True)

In [30]:
stock_price = pd.read_csv("../data/stock_price.csv", parse_dates=[['DATE', 'TIME_M']])

In [31]:
## Interpolation of missing values
idx = pd.date_range(stock_price.iloc[0, 0], stock_price.iloc[-1, 0], freq='1min')
stock_price.index = pd.DatetimeIndex(stock_price.loc[:, 'DATE_TIME_M'])
stock_price = stock_price.reindex(idx, fill_value='NaN')
stock_price['DATE_TIME_M'] = stock_price.index

In [32]:
stock_price = stock_price.astype({'PRICE': 'float'})
stock_price['PRICE'].interpolate(method='linear', inplace=True)

In [33]:
stock_price = stock_price.between_time(datetime.time(4), datetime.time(20), include_start=True, include_end=True)
stock_price_before_merge = stock_price.copy()
stock_price_before_merge.to_csv("../data/stock_price_cleaned.csv", index=False, header=True)

### Combining Stock Data & Tweets

In [21]:
trump_tweets = pd.read_csv("../data/trump_tweets_cleaned_for_lstm.csv")
stock_price = pd.read_csv("../data/stock_price_cleaned.csv")

In [34]:
stock_price.rename(columns={"DATE_TIME_M": "datetime", "PRICE": "price"}, inplace=True)
stock_price['datetime']= pd.to_datetime(stock_price['datetime'])

#### Get Stock Price Difference X mins after

In [60]:
def get_stock_price_x_mins_after(trump_tweets, stock_price, window):
    for i,x_mins in enumerate(window):
        stock_price[f"datetime_{x_mins}mins_before"] = stock_price["datetime"] - pd.Timedelta(f"{x_mins} min")
        stock_price_merged = stock_price.merge(stock_price, how='inner', left_on=[f"datetime_{x_mins}mins_before"], right_on=["datetime"])
        stock_price_merged.drop(columns=["datetime_y", f"datetime_{x_mins}mins_before_y"], inplace=True)
        stock_price_merged.rename(columns={"datetime_x": f"datetime_{x_mins}mins_after", "price_x": f"price_{x_mins}mins_after", f"datetime_{x_mins}mins_before_x": "datetime_now", "price_y": "price_now"}, inplace=True)

        stock_price_merged[f'{x_mins}mins_price_diff_abs'] = stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']
        stock_price_merged[f'{x_mins}mins_price_diff_perc'] = (stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']) / stock_price_merged['price_now']

        trump_tweets.tweet_datetime = pd.to_datetime(trump_tweets.tweet_datetime)
        stock_price_merged.datetime_now = pd.to_datetime(stock_price_merged.datetime_now)
        if i == 0:
            merged = trump_tweets.merge(stock_price_merged, how='inner', left_on=['tweet_datetime'], right_on=["datetime_now"])
        else:
            merged = merged.merge(stock_price_merged, how='inner', left_on=['tweet_datetime'], right_on=["datetime_now"])
    
    return pd.DataFrame(merged)

merged = get_stock_price_x_mins_after(trump_tweets, stock_price, [60, 45, 30, 15])


In [63]:
merged = merged.drop(['datetime_15mins_after', 'price_15mins_after', 'datetime_now_y', 'datetime_30mins_before_x', 'datetime_45mins_before_x', 'datetime_60mins_before_x', 'price_now_y', 'datetime_30mins_before_y', 
                      'datetime_45mins_before_y', 'datetime_60mins_before_y', 'datetime_60mins_after', 'price_60mins_after', 'datetime_15mins_before_x_x', 'datetime_30mins_before_x_x', 'datetime_45mins_before_x_x', 'datetime_now_x', 'price_now_x', 
                      'datetime_15mins_before_y_x','datetime_30mins_before_y_x', 'datetime_45mins_before_y_x', 'datetime_45mins_after', 'price_45mins_after', 'datetime_15mins_before_x_y', 'datetime_30mins_before_x_y', 
                      'datetime_now_y', 'datetime_60mins_before_x_x', 'price_now_y', 'datetime_15mins_before_y_y', 'datetime_30mins_before_y_y', 'datetime_60mins_before_y_x', 'datetime_30mins_after', 'price_30mins_after', 
                      'datetime_15mins_before_x', 'datetime_now_x', 'datetime_45mins_before_x_y', 'datetime_60mins_before_x_y', 'price_now_x', 'datetime_15mins_before_y', 'datetime_45mins_before_y_y', 'datetime_60mins_before_y_y'], axis=1)

merged.to_csv("../data/master_data.csv", index=False, header=True)

['id', 'text', 'favorites', 'retweets', 'date', 'tweet_datetime', 'date_part', 'time_part', 'hour', 'year', 'month', 'text_topic_modelling', 'text_EDA', 'text_sentiment_analysis', 'text_LSTM', '60mins_price_diff_abs', '60mins_price_diff_perc', '45mins_price_diff_abs', '45mins_price_diff_perc', '30mins_price_diff_abs', '30mins_price_diff_perc', '15mins_price_diff_abs', '15mins_price_diff_perc']


### Get Array of Stock Prices 30 minutes before, ie. [-30, -29, -28, ...]

In [69]:
def get_array_of_prices_xmin_before(datetime_now, x_min, stock_price_merged):
    earliest_time = (datetime_now - timedelta(minutes = x_min)).time()
    if earliest_time >= datetime.time(4, 0):
        result = []
        for min_interval in range(x_min, 0, -1):
            datetime_query = datetime_now - timedelta(minutes = min_interval)
            price = stock_price_merged[stock_price_merged['DATE_TIME_M'] == datetime_query].iloc[0, stock_price_merged.columns.get_loc('PRICE')]
            result.append(price)
        return result

In [70]:
merged['prev_30mins_prices'] = merged['tweet_datetime'].apply(get_array_of_prices_xmin_before, args=(30, stock_price_before_merge))

<class 'pandas.core.frame.DataFrame'>


In [71]:
merged.to_csv("../data/data_LSTM_model_b.csv", index=False, header=True)